### This makes margins larger

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


C:\Users\adidr\AppData\Local\Temp\ipykernel_17260\3391910190.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### Connect to IB

In [2]:
from ib_insync import *
ib = IB()
util.startLoop()
ib.connect(port=4002,clientId=0)

<IB connected to 127.0.0.1:4002 clientId=0>

### Define the contract

In [3]:
symbol='TSLA'
contract = Stock(symbol=symbol,exchange='SMART',currency='USD')
ib.qualifyContracts(contract)

[Stock(conId=76792991, symbol='TSLA', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='TSLA', tradingClass='NMS')]

### Historical Bars with Live Updates
* Issue is it becomes useless when you have a disconnection

In [4]:
historical_bars_with_keep_up_to_date = ib.reqHistoricalData(contract=contract,endDateTime='',barSizeSetting='15 secs',durationStr='1 D',whatToShow='TRADES',useRTH=False,keepUpToDate=True)

In [5]:
util.df(historical_bars_with_keep_up_to_date)

,date,open,high,low,close,volume,average,barCount
0,2024-01-31 04:00:00-05:00,185.95,186.20,185.25,185.54,7330.0,185.748000,28
1,2024-01-31 04:00:15-05:00,185.78,185.89,185.78,185.78,300.0,185.817000,3
2,2024-01-31 04:00:30-05:00,185.75,185.75,185.51,185.51,655.0,185.620000,4
3,2024-01-31 04:00:45-05:00,185.58,185.58,185.32,185.32,596.0,185.372000,5
4,2024-01-31 04:01:00-05:00,185.50,186.00,185.40,186.00,1576.0,185.742000,7
...,...,...,...,...,...,...,...,...
1995,2024-01-31 12:18:45-05:00,190.70,190.71,190.51,190.58,58129.0,190.594000,198
1996,2024-01-31 12:19:00-05:00,190.57,190.69,190.54,190.67,25403.0,190.614000,96
1997,2024-01-31 12:19:15-05:00,190.68,190.77,190.65,190.76,26370.0,190.712000,75
1998,2024-01-31 12:19:30-05:00,190.76,190.83,190.72,190.75,27182.0,190.767601,85


### Define onBarUpdate callback

In [6]:
def on_bar_update(bars,hasNewBar):
    print(hasNewBar,bars[-1],len(bars))
historical_bars_with_keep_up_to_date.updateEvent.clear()
historical_bars_with_keep_up_to_date.updateEvent+=on_bar_update

False BarData(date=datetime.datetime(2024, 1, 31, 12, 20, 45, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=190.79, high=190.82, low=190.75, close=190.79, volume=12101.0, average=190.7869010825552, barCount=54) 2004
True BarData(date=datetime.datetime(2024, 1, 31, 12, 21, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=190.82, high=190.82, low=190.82, close=190.82, volume=-1.0, average=190.79, barCount=-1) 2005
False BarData(date=datetime.datetime(2024, 1, 31, 12, 21, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=190.84, high=190.89, low=190.79, close=190.89, volume=21611.0, average=190.8360173060016, barCount=72) 2005
False BarData(date=datetime.datetime(2024, 1, 31, 12, 21, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=190.84, high=190.89, low=190.79, close=190.84, volume=27177.0, average=190.840358759245, barCount=90) 2005
False BarData(date=datetime.datetime(2024, 1, 31, 12, 21, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=190.84, high=190.89, low=190.75, close=1

Error 10182, reqId 164: Failed to request live updates (disconnected).


### Live Bars Robust to Connection Lost
* They backfill

In [8]:
live_bars=ib.reqRealTimeBars(contract=contract,barSize=5,whatToShow="TRADES",useRTH=False)

In [12]:
util.df(live_bars)

,time,endTime,open_,high,low,close,volume,wap,count
0,2024-01-31 17:23:10+00:00,-1,190.73,190.73,190.70,190.73,3525.0,190.715010,12
1,2024-01-31 17:23:15+00:00,-1,190.71,190.73,190.69,190.71,4142.0,190.712586,22
2,2024-01-31 17:23:20+00:00,-1,190.70,190.70,190.63,190.64,3257.0,190.662343,16
3,2024-01-31 17:23:25+00:00,-1,190.65,190.65,190.61,190.64,5041.0,190.639724,27
4,2024-01-31 17:23:30+00:00,-1,190.65,190.70,190.63,190.63,8216.0,190.653236,31
5,2024-01-31 17:23:35+00:00,-1,190.61,190.66,190.61,190.64,7271.0,190.639409,24


In [13]:
def on_bar_update(bars,hasNewBar):
    print(hasNewBar,bars[-1],len(bars))
live_bars.updateEvent.clear()
live_bars.updateEvent+=on_bar_update

True RealTimeBar(time=datetime.datetime(2024, 1, 31, 17, 23, 55, tzinfo=datetime.timezone.utc), endTime=-1, open_=190.69, high=190.7, low=190.66, close=190.66, volume=2800.0, wap=190.6771785714286, count=19) 10
True RealTimeBar(time=datetime.datetime(2024, 1, 31, 17, 24, tzinfo=datetime.timezone.utc), endTime=-1, open_=190.65, high=190.66, low=190.63, close=190.65, volume=3125.0, wap=190.6503232, count=13) 11


In [15]:
live_bars.updateEvent.clear()

### Asside on Pandas Resample

In [17]:
util.df(live_bars).set_index('time').resample('30s').agg(High=('high','max'))

,High
time,
2024-01-31 17:23:00+00:00,190.73
2024-01-31 17:23:30+00:00,190.71
2024-01-31 17:24:00+00:00,190.76
2024-01-31 17:24:30+00:00,190.83
2024-01-31 17:25:00+00:00,190.85
2024-01-31 17:25:30+00:00,191.00
2024-01-31 17:26:00+00:00,191.00
2024-01-31 17:26:30+00:00,190.89
2024-01-31 17:27:00+00:00,190.89


#### Something to be careful about
* Resampling over few days you get Nan it is just a giant mess cuz some days have different regular/regular + extended trading hours due to holidays so make sure you only apply resampling to new liev bar updates

In [29]:
historical_bars = ib.reqHistoricalData(contract=contract,endDateTime='',barSizeSetting='1 hour',durationStr='3 D',whatToShow='TRADES',useRTH=False,keepUpToDate=False)

In [30]:
util.df(historical_bars).set_index('date').resample('2H').agg(High=('high','max'))

,High
date,
2024-01-29 04:00:00-05:00,186.00
2024-01-29 06:00:00-05:00,185.75
2024-01-29 08:00:00-05:00,186.95
2024-01-29 10:00:00-05:00,190.90
2024-01-29 12:00:00-05:00,189.45
2024-01-29 14:00:00-05:00,191.48
2024-01-29 16:00:00-05:00,192.21
2024-01-29 18:00:00-05:00,192.60
2024-01-29 20:00:00-05:00,NaN


### Need to start real time bars before last timestamp in historical bars
* Use historical bars up until last timestamp before live bars start
* This is why I recommend starting your algo before pre-market if you use premarket data and before market if you use regular trading hours only if your bar size is large and make sure to define live bars before premarket/market start respectively.
* If it is small you can just used second to last historical bar and it will be fine, but make sure to start live bars before historical bars. 

* This is because historical bars gives you latest bar even if not completely


In [22]:
historical_bars = ib.reqHistoricalData(contract=contract,endDateTime='',barSizeSetting='1 hour',durationStr='3 D',whatToShow='TRADES',useRTH=False,keepUpToDate=False)
util.df(historical_bars)

,date,open,high,low,close,volume,average,barCount
0,2024-01-29 04:00:00-05:00,185.55,186.00,184.70,185.00,93113.0,185.329,481
1,2024-01-29 05:00:00-05:00,184.91,185.48,184.81,185.20,35275.0,185.330,194
2,2024-01-29 06:00:00-05:00,185.23,185.70,185.21,185.30,47743.0,185.483,237
3,2024-01-29 07:00:00-05:00,185.32,185.75,184.88,185.10,175094.0,185.299,781
4,2024-01-29 08:00:00-05:00,185.57,186.95,184.75,184.83,495915.0,185.301,1888
5,2024-01-29 09:00:00-05:00,184.80,186.88,183.67,184.10,14548114.0,185.122,48629
6,2024-01-29 10:00:00-05:00,184.10,188.42,183.73,187.29,19811842.0,186.200,69156
7,2024-01-29 11:00:00-05:00,187.27,190.90,187.20,189.38,20869363.0,189.464,73494
8,2024-01-29 12:00:00-05:00,189.43,189.45,187.62,188.40,11764414.0,188.321,39371
9,2024-01-29 13:00:00-05:00,188.41,188.60,187.20,188.60,8546354.0,187.862,27677


### Create a function to update historical bars with real time bars

### Need to match timezones

In [26]:
util.df(live_bars).set_index('time').tz_convert('US/Eastern')

,endTime,open_,high,low,close,volume,wap,count
time,,,,,,,,
2024-01-31 12:23:10-05:00,-1,190.73,190.73,190.70,190.73,3525.0,190.715010,12
2024-01-31 12:23:15-05:00,-1,190.71,190.73,190.69,190.71,4142.0,190.712586,22
2024-01-31 12:23:20-05:00,-1,190.70,190.70,190.63,190.64,3257.0,190.662343,16
2024-01-31 12:23:25-05:00,-1,190.65,190.65,190.61,190.64,5041.0,190.639724,27
2024-01-31 12:23:30-05:00,-1,190.65,190.70,190.63,190.63,8216.0,190.653236,31
...,...,...,...,...,...,...,...,...
2024-01-31 12:32:15-05:00,-1,190.70,190.82,190.67,190.82,28753.0,190.698270,38
2024-01-31 12:32:20-05:00,-1,190.82,190.83,190.80,190.83,5732.0,190.818222,13
2024-01-31 12:32:25-05:00,-1,190.81,190.89,190.80,190.81,9938.0,190.858153,48


In [28]:
util.df(historical_bars).set_index('date').columns[:-1]

Index(['open', 'high', 'low', 'close', 'volume', 'average'], dtype='object')

### Function to update historical bars with real time updates
* Assuming historical df in Eastern Time!!!
* Use historical_bars_from_dayb4= True if you started the reqRealTimeBars before either market/premarket else use False

In [45]:
import pandas as pd
def update_df(historical_df,live_bars,bar_size,historical_bars_from_dayb4=False):
    # Define column names
    col_names=['open', 'high', 'low', 'close', 'volume', 'average']
    
    # Get the dataframe and set the index to the timestamp and get necessary columns
    df_live_bars = util.df(live_bars).set_index('time')[['open_', 'high', 'low', 'close', 'volume', 'wap']]
    
    #For calculating Average Price
    df_live_bars['average_times_volume']=df_live_bars['volume']*df_live_bars['wap']
        
    # Make timezones the same
    df_live_bars=df_live_bars.tz_convert('US/Eastern')
    
    # Resample the live bars
    df_live_bars_resampled = df_live_bars.resample(bar_size).agg(
    open = ('open_','first'),
    high = ('high','max'),
    low = ('low','min'),
    close = ('close','last'),
    volume = ('volume','sum'),
    average_times_volume = ('average_times_volume','sum')
    )
    
    # Calculate average price
    df_live_bars_resampled['average']=df_live_bars_resampled['average_times_volume']/df_live_bars_resampled['volume']
    
    if historical_bars_from_dayb4:
        df_combined = pd.concat([historical_df[col_names], df_live_bars_resampled[col_names]])
    else: #otherwise we must make sure our last historical bar is a full interval!!!
        df_combined = pd.concat([historical_df[col_names].iloc[:-1,:], df_live_bars_resampled[col_names].loc[historical_df.index[-1]:]])
    return df_combined

### Examples 15 sec bars

In [46]:
historical_bars = ib.reqHistoricalData(contract=contract,endDateTime='',barSizeSetting='15 secs',durationStr='1 D',whatToShow='TRADES',useRTH=False,keepUpToDate=False)

In [47]:
historical_df = util.df(historical_bars).set_index('date')

In [48]:
update_df(historical_df, live_bars,bar_size='15s')

,open,high,low,close,volume,average
2024-01-31 04:00:00-05:00,185.95,186.20,185.25,185.54,7330.0,185.748000
2024-01-31 04:00:15-05:00,185.78,185.89,185.78,185.78,300.0,185.817000
2024-01-31 04:00:30-05:00,185.75,185.75,185.51,185.51,655.0,185.620000
2024-01-31 04:00:45-05:00,185.58,185.58,185.32,185.32,596.0,185.372000
2024-01-31 04:01:00-05:00,185.50,186.00,185.40,186.00,1576.0,185.742000
...,...,...,...,...,...,...
2024-01-31 12:46:00-05:00,191.47,191.54,191.46,191.49,58556.0,191.499000
2024-01-31 12:46:15-05:00,191.50,191.50,191.37,191.45,58282.0,191.448000
2024-01-31 12:46:30-05:00,191.43,191.48,191.34,191.42,43376.0,191.413000
2024-01-31 12:46:45-05:00,191.42,191.45,191.35,191.39,40706.0,191.403000


### Make it a callback

In [50]:
from IPython.display import display,clear_output
bar_size='15s'
def on_bar_update(bars,hasNewBar):
     global bar_size
     df=update_df(historical_df,bars,bar_size,historical_bars_from_dayb4=False)
     clear_output()
     display(df.tail())
live_bars.updateEvent.clear()
live_bars.updateEvent+=on_bar_update

,open,high,low,close,volume,average
2024-01-31 12:49:15-05:00,191.46,191.49,191.37,191.37,21202.0,191.444473
2024-01-31 12:49:30-05:00,191.38,191.54,191.37,191.52,21645.0,191.447314
2024-01-31 12:49:45-05:00,191.52,191.60,191.49,191.55,56628.0,191.525114
2024-01-31 12:50:00-05:00,191.54,191.58,191.52,191.55,22430.0,191.549578
2024-01-31 12:50:15-05:00,191.55,191.58,191.53,191.55,16987.0,191.557105
